In [1]:
import wandb
import pandas as pd
import pandas_profiling

In [2]:
run = wandb.init(project="nyc_airbnb", group="eda", save_code=True)
local_path = wandb.use_artifact("sample.csv:latest").file()
df = pd.read_csv(local_path)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nhattran134. Use `wandb login --relogin` to force relogin


In [3]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,9138664,Private Lg Room 15 min to Manhattan,47594947,Iris,Queens,Sunnyside,40.74271,-73.92493,Private room,74,2,6,2019-05-26,0.13,1,5
1,31444015,TIME SQUARE CHARMING ONE BED IN HELL'S KITCHEN...,8523790,Johlex,Manhattan,Hell's Kitchen,40.76682,-73.98878,Entire home/apt,170,3,0,NaN,NaN,1,188
2,8741020,Voted #1 Location Quintessential 1BR W Village...,45854238,John,Manhattan,West Village,40.73631,-74.00611,Entire home/apt,245,3,51,2018-09-19,1.12,1,0
3,34602077,Spacious 1 bedroom apartment 15min from Manhattan,261055465,Regan,Queens,Astoria,40.76424,-73.92351,Entire home/apt,125,3,1,2019-05-24,0.65,1,13
4,23203149,Big beautiful bedroom in huge Bushwick apartment,143460,Megan,Brooklyn,Bushwick,40.69839,-73.92044,Private room,65,2,8,2019-06-23,0.52,2,8


In [4]:
profile = pandas_profiling.ProfileReport(df)
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,neighbourhood_group,room_type
id,1.000000,0.558145,-0.001193,0.066518,-0.026606,-0.068274,-0.302146,0.368858,0.133321,0.164731,0.059137,0.070785
host_id,0.558145,1.000000,0.041903,0.106627,-0.073509,-0.138851,-0.117507,0.275662,0.144830,0.172829,0.098353,0.092035
latitude,-0.001193,0.041903,1.000000,0.040318,0.138701,0.017124,-0.038288,-0.027334,0.002377,-0.009651,0.538339,0.111624
longitude,0.066518,0.106627,0.040318,1.000000,-0.436764,-0.121498,0.080778,0.117056,0.062861,0.061113,0.648797,0.153825
price,-0.026606,-0.073509,0.138701,-0.436764,1.000000,0.103245,-0.050723,-0.018707,-0.101398,0.091799,0.010046,0.022859
minimum_nights,-0.068274,-0.138851,0.017124,-0.121498,0.103245,1.000000,-0.159285,-0.288277,0.063765,0.081264,0.000000,0.020005
number_of_reviews,-0.302146,-0.117507,-0.038288,0.080778,-0.050723,-0.159285,1.000000,0.701699,0.057464,0.237886,0.026979,0.000000
reviews_per_month,0.368858,0.275662,-0.027334,0.117056,-0.018707,-0.288277,0.701699,1.000000,0.144355,0.387509,0.065914,0.017332
calculated_host_listings_count,0.133321,0.144830,0.002377,0.062861,-0.101398,0.063765,0.057464,0.144355,1.000000,0.407154,0.086084,0.095414
availability_365,0.164731,0.172829,-0.009651,0.061113,0.091799,0.081264,0.237886,0.387509,0.407154,1.000000,0.075558,0.088758


In [5]:
# Drop outliers
min_price = 10
max_price = 350
idx = df['price'].between(min_price, max_price)
df = df[idx].copy()
# Convert last_review to datetime
df['last_review'] = pd.to_datetime(df['last_review'])

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19001 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              19001 non-null  int64         
 1   name                            18994 non-null  object        
 2   host_id                         19001 non-null  int64         
 3   host_name                       18993 non-null  object        
 4   neighbourhood_group             19001 non-null  object        
 5   neighbourhood                   19001 non-null  object        
 6   latitude                        19001 non-null  float64       
 7   longitude                       19001 non-null  float64       
 8   room_type                       19001 non-null  object        
 9   price                           19001 non-null  int64         
 10  minimum_nights                  19001 non-null  int64         
 11  nu

Check null percentage

In [7]:
df.isna().sum()/len(df)

id                                0.000000
name                              0.000368
host_id                           0.000000
host_name                         0.000421
neighbourhood_group               0.000000
neighbourhood                     0.000000
latitude                          0.000000
longitude                         0.000000
room_type                         0.000000
price                             0.000000
minimum_nights                    0.000000
number_of_reviews                 0.000000
last_review                       0.197779
reviews_per_month                 0.197779
calculated_host_listings_count    0.000000
availability_365                  0.000000
dtype: float64

In [8]:
run.finish()